## CS324 Introduction Assignment

### Overview

In this assignment, you will gain familiarity with prompting a wide range of foundation models. Specifically, you’ll work with large language models (LLMs) such as [BLOOM](https://huggingface.co/bigscience/bloom) and [BLOOMZ](https://huggingface.co/bigscience/bloomz) (BigScience). Language model prompting is the process of providing a model with an input text, and then having the model generate a response as output. For example, one could provide a language model with the prompt "The sky is" and the model might generate the response "blue". 

Prompts can be used to complete a breadth of tasks such as summarization (e.g., “Summarize the following paragraph: \<paragraph written out here\> Summary:”) or extraction (e.g., “Extract the phone number from the user bio: \<bio written out here\>.”)


The assignment is divided into three parts:


1. **Task selection**: In the first part of the assignment, you will select a task and evaluation dataset, which you will use in the remainder of the assignment.

2. **Prompt development**: In the second part of the assignment, you will explore how to develop effective prompts for LLMs. As part of this process, you may encounter both the joys and pains of prompting.  
  * On one hand, the right prompt and model can magically solve a seemingly complicated and unrelated task (w.r.t. the model's *training objective*.)  
  * On the other, prompting can be a brittle technique, and it may require time, effort, and creativity to develop a sufficient prompt.

3. **Model comparisons**: In the final part of the assignment, you will compare the performance (qualitatively and quantitatively) of several LLMs. Model comparisons will be along two dimensions:

  * Model Size
  * Training Techniques (e.g., vanilla autoregressive vs. instruction-tuned) 

### Submission Instructions

Complete the following sections. Download and submit this notebook (as an .ipynb file) to Canvas by 11:59PM on 1/24/23. 


---
### Part 0: Notebook Set-up

##### 0. Connect to a GPU supported runtime.

Connect to a hosted runtime and change the runtime type to "GPU".

<!-- Runtime -> Change Runtime Type -> Select "GPU" under Hardware Accelerator -->

##### 1. Install dependencies
First, make sure you have the necessary dependencies installed. You will need to install the transformers and torch libraries, as well as the huggingface-datasets library. You can do this by running the following commands:

In [ ]:
!pip install transformers torch datasets  accelerate bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##### 2. Import necessary libraries

Next, import the necessary libraries into your Jupyter notebook. You will need to import the following libraries

In [ ]:
import transformers
import torch
from datasets import load_dataset
from transformers import pipeline

---
### Part 1: Task, dataset, and evaluation metric selection [5 points]

In the first part of the assignment, you will select a task (e.g., sentiment classification) which you will use for the remainder of this assignment. Use the dataset and tasks found in the HuggingFace datasets library (https://huggingface.co/datasets). Select a labeled dataset (e.g., IMDB sentiment classification) and identify an evaluation metric (e.g., F1-score). 
 

In [ ]:
""" Example code to load dataset """
sample_dataset = load_dataset("imdb", split="test")

**Load your dataset below**

In [ ]:
# dataset = load_dataset("SetFit/tweet_sentiment_extraction", split="test")
dataset = load_dataset("mteb/tweet_sentiment_extraction", split="test")

In 2-3 lines, describe your chosen task, dataset and evaluation metric:

**Answer**: 

- I decided to use the tween sentiment extraction analysis.
- eval - simple 0-1 accuracy, I implemented it. It's simple.

---
### Part 2: Prompt Development [30 points]

In the second part of the assignment, you will develop effective prompts for the foundation model. A prompt is a short sentence or phrase that serves as the starting point for the model's text generation (e.g., “Is the review positive or negative? \<review written here\>”). 

You will try to come up with a prompting "template" that allows a foundation model to correctly solve a task for 5 samples of your selected dataset.

For example, for sentiment classification of clothing reviews, we could specify a template as follows: 
```python
def template(sample_review):
    prompt = f"""
    Is this review positive or negative: 
    "Perfect dress and the customer service was awesome!"
    Sentiment: 
    positive
    ###
    Is this review positive or negative: 
    "not exact in sizing"
    Sentiment: 
    negative
    ###
    {sample_review}
    Sentiment:
    """
    return prompt
```
We can then apply the `template` function to 5 samples of our dataset, and feed the resulting prompt to a large language model. The hope is that the LLM processes the prompt as input, and outputs the desired `positive` or `negative` classification as natural text, via standard autoregressive language modeling.  


**To begin**, first select 5 samples from your chosen dataset.

In this section we will develop prompts for HuggingFace Bloom's 1.7B parameter model (a large language model with... 1.7 billion parameters). 

We'll first walk through some initial setup and two prompting strategies, before you come up with your own templates.

#### 1. Load the model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model_name = "bigscience/bloom-1b7"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_name)
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
For effortless bug reporting copy-paste your error into this form: https://docs.google.com/forms/d/e/1FAIpQLScPB8emS3Thkp66nvqwmjTEgxp8Y9ufuWTzFyr9kJ5AoI47dQ/viewform?usp=sf_link
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 112
CUDA SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda112.so...


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:134: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:134: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:134: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/share/tcltk/tcllib1.19')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:134: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28.0.12 --tunnel_background_save_url=https'), PosixPath('//colab.resear

#### 2. Configure a HuggingFace pipeline to generate text

Here, we will use the HuggingFace pipeline library to generate text using the model we have loaded

In [ ]:
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

#### 3. Generate Text 
Now we will now begin prompting the model. We will begin by walking you through a few examples of different prompt formats.

In [ ]:
# sample five entries from the sample dataset
samples = sample_dataset.to_pandas().sample(n=5)
sample_review = samples.iloc[0].text
print(sample_review)

This film offers many delights and surprises. When Achille and Philippa beautifully sing a duet from "Don Giovanni" that perfectly describes their situation in the movie, you appreciate the subtle layers of this excellent film. The story unfolds in 18th century Jutland and the use of period music played on period instruments is just one more fine touch. You share General Loewenhielm's exquisite joy in his partaking of the Cailles en Sarcophage even though you are just watching a movie - but you do wish for just a small sample to savor.<br /><br />Babette is an artist whose medium is food. Perhaps no other art form allows the artist to share her creations so directly.<br /><br />The main theme of this movie, the potential that the sharing of food has to transform how people see each other and how they see the world, is much the same as the theme of "Chololat," but "Babette's Feast" does not hit you over the head with its message. The townspeople are conservative puritans, but not exagge

#### 3.1 Prompt Technique: Zero-shot prompting

In this technique, an instruction for the task is usually specified in natural language. The model is expected to following the specification and output a correct response, **without any examples** (hence "zero shots").

In [ ]:
# Write a zero-shot prompt
prompt = f"""Classify the following movie review as positive or negative

Review: {sample_review}
Sentiment:"""

print(prompt)

Classify the following movie review as positive or negative

Review: This film offers many delights and surprises. When Achille and Philippa beautifully sing a duet from "Don Giovanni" that perfectly describes their situation in the movie, you appreciate the subtle layers of this excellent film. The story unfolds in 18th century Jutland and the use of period music played on period instruments is just one more fine touch. You share General Loewenhielm's exquisite joy in his partaking of the Cailles en Sarcophage even though you are just watching a movie - but you do wish for just a small sample to savor.<br /><br />Babette is an artist whose medium is food. Perhaps no other art form allows the artist to share her creations so directly.<br /><br />The main theme of this movie, the potential that the sharing of food has to transform how people see each other and how they see the world, is much the same as the theme of "Chololat," but "Babette's Feast" does not hit you over the head with i

In [ ]:
# Feed prompt to model to generate an output
output = generator(prompt, max_new_tokens=20)
print(output[0]['generated_text'])

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1359: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Classify the following movie review as positive or negative

Review: This film offers many delights and surprises. When Achille and Philippa beautifully sing a duet from "Don Giovanni" that perfectly describes their situation in the movie, you appreciate the subtle layers of this excellent film. The story unfolds in 18th century Jutland and the use of period music played on period instruments is just one more fine touch. You share General Loewenhielm's exquisite joy in his partaking of the Cailles en Sarcophage even though you are just watching a movie - but you do wish for just a small sample to savor.<br /><br />Babette is an artist whose medium is food. Perhaps no other art form allows the artist to share her creations so directly.<br /><br />The main theme of this movie, the potential that the sharing of food has to transform how people see each other and how they see the world, is much the same as the theme of "Chololat," but "Babette's Feast" does not hit you over the head with i

#### 3.1 Prompt Technique: Few-shot prompting

In this technique, we provide a **few examples** in the prompt, optionally including task instructions as well (all as natural language). Even without said instructions, our hope is that the LLM can use the examples to autoregressively complete what comes next to solve the desired task. 


In [ ]:
""" 
Write a few-shot prompt. Here we include a few in-context examples to the model 
demonstrating how to complete the tasks
"""

prompt = f"""Review: The movie was horrible
Sentiment: Negative

Review: The movie was the best movie I have watched all year!!!
Sentiment: Positive

Review: {sample_review}
Sentiment:"""

print(prompt)

Review: The movie was horrible
Sentiment: Negative

Review: The movie was the best movie I have watched all year!!!
Sentiment: Positive

Review: This film offers many delights and surprises. When Achille and Philippa beautifully sing a duet from "Don Giovanni" that perfectly describes their situation in the movie, you appreciate the subtle layers of this excellent film. The story unfolds in 18th century Jutland and the use of period music played on period instruments is just one more fine touch. You share General Loewenhielm's exquisite joy in his partaking of the Cailles en Sarcophage even though you are just watching a movie - but you do wish for just a small sample to savor.<br /><br />Babette is an artist whose medium is food. Perhaps no other art form allows the artist to share her creations so directly.<br /><br />The main theme of this movie, the potential that the sharing of food has to transform how people see each other and how they see the world, is much the same as the them

In [ ]:
# Feed prompt to model to generate an output
output = generator(prompt, max_new_tokens=1)
print(output[0]['generated_text'])

Review: The movie was horrible
Sentiment: Negative

Review: The movie was the best movie I have watched all year!!!
Sentiment: Positive

Review: This film offers many delights and surprises. When Achille and Philippa beautifully sing a duet from "Don Giovanni" that perfectly describes their situation in the movie, you appreciate the subtle layers of this excellent film. The story unfolds in 18th century Jutland and the use of period music played on period instruments is just one more fine touch. You share General Loewenhielm's exquisite joy in his partaking of the Cailles en Sarcophage even though you are just watching a movie - but you do wish for just a small sample to savor.<br /><br />Babette is an artist whose medium is food. Perhaps no other art form allows the artist to share her creations so directly.<br /><br />The main theme of this movie, the potential that the sharing of food has to transform how people see each other and how they see the world, is much the same as the them

#### **Now it's your turn!** 

Create and evaluate 3 different prompting templates for your task and the 5 previously selected samples. 

For each sample, your code should print out the LLM's output in response to the sample's corresponding prompt. 

You should also write a function to automatically determine if the output correctly solved the task for each sample, and report the overall evaluation metric for all five samples.

Don't worry about creating templates that solve the task for all samples (e.g., 100% accuracy for classification). But your templates should be reasonable to some extent, i.e., developed with the aim to solve the task for at least 3 or 4 examples.

In [ ]:
"""
Instructions: Randomly subselect 5 examples from your dataset

Enter your code below:
"""

samples = dataset.to_pandas().sample(n=5)
sample_review = samples.iloc[0].text
print(sample_review)

Unlike cierra, i look like poop today. Whatevahh, lol.


In [ ]:
"""
Prompt #1: Enter your code below

** Hint: For ease of use, wrap your prompts in functions (see sample code below) ***

----- SAMPLE CODE ------

def prompt_1(review):
  prompt = f'''Review: The movie was horrible
  Sentiment: Negative

  Review: The movie was the best movie I have watched all year!!!
  Sentiment: Positive

  Review: The film was a disaster
  Sentiment: Negative

  Review: {review}
  Sentiment:'''
  return prompt

for i, row in samples.iterrows():
  print(f"Sample {i}:")
  print(generator(prompt_1(row['text']), max_new_tokens=1)[0]['generated_text'])

-------------------------

"""
# changed from Neutral to neutral do match output distribution of data, now it predicts the full word
def prompt_1(tweet):
  # prompt = f'''Tweet: Sorry to anyone still stuck at Google Gaslight, where they repeatedly tell you it's the best place to work or whatever while they cull your colleagues by cutting off Corp access abruptly at 2AM
  # Sentiment: negative

  # Tweet:  think I'm going to succeed as a professor and, like, eventually get tenure. The first couple of conference and journal paper acceptances as junior faculty are a wild confidence booster
  # Sentiment: positive

  # Tweet: I saw a bird today at 4pm in the afernoon.
  # Sentiment: neutral

  # Tweet: {tweet}
  # Sentiment:'''
  # prompt = f'''Tweet: On the way to Malaysia...no internet access to Twit
  # Sentiment: negative

  # Tweet: juss came backk from Berkeleyy ; omg its madd fun out there havent been out there in a minute . whassqoodd ?
  # Sentiment: positive

  # Tweet: I saw a bird today at 4pm in the afernoon.
  # Sentiment: neutral

  # Tweet: {tweet}
  # Sentiment:'''
  prompt = f'''Tweet: just arrived home and found it TRASHED, this going to be so costly
  Sentiment: negative

  Tweet: I am full of hope, I can go to work today and have a job
  Sentiment: positive

  Tweet: I saw a bird today at 4pm in the afernoon.
  Sentiment: neutral

  Tweet: You are the only person who is walking around #walking
  Sentiment: neutral

  Tweet: {tweet}
  Sentiment:'''
  return prompt

# do assignment once
import random
import numpy as np

seed = 0
random.seed(seed)
np.random.seed(seed)

def get_answer_from_fm(generated_text) -> str:
  gen_ans = generated_text.split('\n')[-1].split(' ')[-1]
  return gen_ans

correct = 0
tot = 0
for i, row in samples.iterrows():
  # print(f'{row.keys()=}')
  print(f'\n\n --- Sample {i=}:')
  #print(f"{row['text']=}")
  generated_text: str = generator(prompt_1(row['text']), max_new_tokens=1)[0]['generated_text']
  print(generated_text)
  gen_ans = get_answer_from_fm(generated_text)
  print(f'{gen_ans=}')
  print(f"correct answer: {row['label_text']}")
  correct += int(gen_ans == row['label_text'])
  tot += 1

acc = correct/tot
print(f'{correct=}, {tot=}, {acc=}')



 --- Sample i=621:
Tweet: just arrived home and found it TRASHED, this going to be so costly
  Sentiment: negative

  Tweet: I am full of hope, I can go to work today and have a job
  Sentiment: positive

  Tweet: I saw a bird today at 4pm in the afernoon.
  Sentiment: neutral

  Tweet: You are the only person who is walking around #walking
  Sentiment: neutral

  Tweet: Unlike cierra, i look like poop today. Whatevahh, lol.
  Sentiment: neutral
gen_ans='neutral'
correct answer: neutral


 --- Sample i=874:
Tweet: just arrived home and found it TRASHED, this going to be so costly
  Sentiment: negative

  Tweet: I am full of hope, I can go to work today and have a job
  Sentiment: positive

  Tweet: I saw a bird today at 4pm in the afernoon.
  Sentiment: neutral

  Tweet: You are the only person who is walking around #walking
  Sentiment: neutral

  Tweet:  Sorry to hear about your dude probs, what is it that`s too much?
  Sentiment: negative
gen_ans='negative'
correct answer: negativ

**Performance of Prompt #1:** 

correct=2, tot=5, acc=0.4

(not too impressive, odd, it was 0.6 yesterday...but my other 2 prompts were much better)

In [ ]:
# do assignment once
import random
import numpy as np

seed = 1
random.seed(seed)
np.random.seed(seed)

"""
Prompt #2: Enter your code below
"""

samples = dataset.to_pandas().sample(n=5)
sample_review = samples.iloc[0].text
print(sample_review)

def prompt_1(tweet):
  prompt = f'''Tweet: Sorry to anyone still stuck at Google Gaslight, where they repeatedly tell you it's the best place to work or whatever while they cull your colleagues by cutting off Corp access abruptly at 2AM
  Sentiment: negative

  Tweet:  think I'm going to succeed as a professor and, like, eventually get tenure. The first couple of conference and journal paper acceptances as junior faculty are a wild confidence booster
  Sentiment: positive

  Tweet: I saw a bird today at 4pm in the afernoon.
  Sentiment: neutral

  Tweet: juss came backk from Berkeleyy ; omg its madd fun out there havent been out there in a minute . whassqoodd ?
  Sentiment: positive

  Tweet: On the way to Malaysia...no internet access to Twit
  Sentiment: negative

  Tweet: is cleaning the house for her family who is comming later today..
  Sentiment: neutral

  Tweet: There won't another challenge really. Thanks.
  Sentiment: neutral

  Tweet: It's fun to go out dancing.
  Sentiment: positive

  Tweet: We will deliver you athletic greens. Information comming soon and will be sent to your email if you signed up.
  Sentiment: neutral

  Tweet: {tweet}
  Sentiment:'''
  return prompt

def get_answer_from_fm(generated_text) -> str:
  gen_ans = generated_text.split('\n')[-1].split(' ')[-1]
  return gen_ans

correct = 0
tot = 0
for i, row in samples.iterrows():
  # print(f'{row.keys()=}')
  print(f'\n\n --- Sample {i=}:')
  #print(f"{row['text']=}")
  generated_text: str = generator(prompt_1(row['text']), max_new_tokens=1)[0]['generated_text']
  print(generated_text)
  gen_ans = get_answer_from_fm(generated_text)
  print(f'{gen_ans=}')
  print(f"correct answer: {row['label_text']}")
  correct += int(gen_ans == row['label_text'])
  tot += 1

acc = correct/tot
print(f'{correct=}, {tot=}, {acc=}')

 Aren`t you suppossed to support the local economy ... LOL ... good for you and congrats


 --- Sample i=161:
Tweet: Sorry to anyone still stuck at Google Gaslight, where they repeatedly tell you it's the best place to work or whatever while they cull your colleagues by cutting off Corp access abruptly at 2AM
  Sentiment: negative

  Tweet:  think I'm going to succeed as a professor and, like, eventually get tenure. The first couple of conference and journal paper acceptances as junior faculty are a wild confidence booster
  Sentiment: positive

  Tweet: I saw a bird today at 4pm in the afernoon.
  Sentiment: neutral

  Tweet: juss came backk from Berkeleyy ; omg its madd fun out there havent been out there in a minute . whassqoodd ?
  Sentiment: positive

  Tweet: On the way to Malaysia...no internet access to Twit
  Sentiment: negative

  Tweet: is cleaning the house for her family who is comming later today..
  Sentiment: neutral

  Tweet: There won't another challenge really. Thank

**Performance of Prompt #2:**

correct=4, tot=5, acc=0.8

In [ ]:
"""
Prompt #3: Enter your code below
"""

# do assignment once
import random
import numpy as np

seed = 2
random.seed(seed)
np.random.seed(seed)

samples = dataset.to_pandas().sample(n=5)
sample_review = samples.iloc[0].text
print(sample_review)

def prompt_1(tweet):
  prompt = f'''Tweet: Sorry to anyone still stuck at Google Gaslight, where they repeatedly tell you it's the best place to work or whatever while they cull your colleagues by cutting off Corp access abruptly at 2AM
  Sentiment: negative

  Tweet:  think I'm going to succeed as a professor and, like, eventually get tenure. The first couple of conference and journal paper acceptances as junior faculty are a wild confidence booster
  Sentiment: positive

  Tweet: I saw a bird today at 4pm in the afernoon.
  Sentiment: neutral

  Tweet: juss came backk from Berkeleyy ; omg its madd fun out there havent been out there in a minute . whassqoodd ?
  Sentiment: positive

  Tweet: On the way to Malaysia...no internet access to Twit
  Sentiment: negative

  Tweet: is cleaning the house for her family who is comming later today..
  Sentiment: neutral

  Tweet: There won't another challenge really. Thanks.
  Sentiment: neutral

  Tweet: It's fun to go out dancing.
  Sentiment: positive

  Tweet: We will deliver you athletic greens. Information comming soon and will be sent to your email if you signed up.
  Sentiment: neutral

  Tweet: LOL. Playing soccer isn`t easy anymore compared when we were young
  Sentiment: negative

  Tweet: {tweet}
  Sentiment:'''
  return prompt

def get_answer_from_fm(generated_text) -> str:
  gen_ans = generated_text.split('\n')[-1].split(' ')[-1]
  return gen_ans

correct = 0
tot = 0
for i, row in samples.iterrows():
  # print(f'{row.keys()=}')
  print(f'\n\n --- Sample {i=}:')
  #print(f"{row['text']=}")
  generated_text: str = generator(prompt_1(row['text']), max_new_tokens=1)[0]['generated_text']
  print(generated_text)
  gen_ans = get_answer_from_fm(generated_text)
  print(f'{gen_ans=}')
  print(f"correct answer: {row['label_text']}")
  correct += int(gen_ans == row['label_text'])
  tot += 1

acc = correct/tot
print(f'{correct=}, {tot=}, {acc=}')

My mobile has stopped recieving text messages AGAIN. Stoooopid thing


 --- Sample i=236:
Tweet: Sorry to anyone still stuck at Google Gaslight, where they repeatedly tell you it's the best place to work or whatever while they cull your colleagues by cutting off Corp access abruptly at 2AM
  Sentiment: negative

  Tweet:  think I'm going to succeed as a professor and, like, eventually get tenure. The first couple of conference and journal paper acceptances as junior faculty are a wild confidence booster
  Sentiment: positive

  Tweet: I saw a bird today at 4pm in the afernoon.
  Sentiment: neutral

  Tweet: juss came backk from Berkeleyy ; omg its madd fun out there havent been out there in a minute . whassqoodd ?
  Sentiment: positive

  Tweet: On the way to Malaysia...no internet access to Twit
  Sentiment: negative

  Tweet: is cleaning the house for her family who is comming later today..
  Sentiment: neutral

  Tweet: There won't another challenge really. Thanks.
  Sentiment: neut

**Performance of Prompt #3:**

correct=4, tot=5, acc=0.8

---
### Part 3. Model comparisons [30 points]
In the final part of the assignment, pick one of the prompting templates you designed in Part 2, and randomly select 20 samples from your dataset chosen in Part 1. 

Using this template and evaluation set, we will now compare the performance of different large language models. 

We will focus on two dimensions of comparisons: 
* Model Size
* Model Training Objectives

#### 1. Model Size Comparisons
Here we will compare the performance of our prior Bloom 1.7b model (`bloom-1b7`), and a larger LLM, [bloom-3B](https://huggingface.co/bigscience/bloom-3b) (`bloom-3b`) with... you guessed it, *3 billion* parameters.

In [ ]:
"""
Instructions: Randomly subselect 20 examples from your dataset

Enter your code below:
"""

"""
Prompt #3: Enter your code below
"""

# do assignment once
import random
import numpy as np

seed = 2
random.seed(seed)
np.random.seed(seed)

samples = dataset.to_pandas().sample(n=20)
sample_review = samples.iloc[0].text
print(f'{sample_review=}')

def prompt_1(tweet):
  prompt = f'''Tweet: Sorry to anyone still stuck at Google Gaslight, where they repeatedly tell you it's the best place to work or whatever while they cull your colleagues by cutting off Corp access abruptly at 2AM
  Sentiment: negative

  Tweet:  think I'm going to succeed as a professor and, like, eventually get tenure. The first couple of conference and journal paper acceptances as junior faculty are a wild confidence booster
  Sentiment: positive

  Tweet: I saw a bird today at 4pm in the afernoon.
  Sentiment: neutral

  Tweet: juss came backk from Berkeleyy ; omg its madd fun out there havent been out there in a minute . whassqoodd ?
  Sentiment: positive

  Tweet: On the way to Malaysia...no internet access to Twit
  Sentiment: negative

  Tweet: is cleaning the house for her family who is comming later today..
  Sentiment: neutral

  Tweet: There won't another challenge really. Thanks.
  Sentiment: neutral

  Tweet: It's fun to go out dancing.
  Sentiment: positive

  Tweet: We will deliver you athletic greens. Information comming soon and will be sent to your email if you signed up.
  Sentiment: neutral

  Tweet: LOL. Playing soccer isn`t easy anymore compared when we were young
  Sentiment: negative

  Tweet: {tweet}
  Sentiment:'''
  return prompt

def get_answer_from_fm(generated_text) -> str:
  gen_ans = generated_text.split('\n')[-1].split(' ')[-1]
  return gen_ans

from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model_name = "bigscience/bloom-1b7"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)
print(f'{model.num_parameters()=}')

tokenizer = AutoTokenizer.from_pretrained(model_name)
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

correct = 0
tot = 0
for i, row in samples.iterrows():
  generated_text: str = generator(prompt_1(row['text']), max_new_tokens=1)[0]['generated_text']
  gen_ans = get_answer_from_fm(generated_text)
  print(f'\n\n --- Sample {i=}:')
  print(generated_text)
  print(f'{gen_ans=}')
  print(f"correct answer: {row['label_text']}")
  correct += int(gen_ans == row['label_text'])
  tot += 1

acc = correct/tot
print(f'{correct=}, {tot=}, {acc=}')

sample_review='My mobile has stopped recieving text messages AGAIN. Stoooopid thing'
model.num_parameters()=1722408960


/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1359: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(




 --- Sample i=236:
Tweet: Sorry to anyone still stuck at Google Gaslight, where they repeatedly tell you it's the best place to work or whatever while they cull your colleagues by cutting off Corp access abruptly at 2AM
  Sentiment: negative

  Tweet:  think I'm going to succeed as a professor and, like, eventually get tenure. The first couple of conference and journal paper acceptances as junior faculty are a wild confidence booster
  Sentiment: positive

  Tweet: I saw a bird today at 4pm in the afernoon.
  Sentiment: neutral

  Tweet: juss came backk from Berkeleyy ; omg its madd fun out there havent been out there in a minute . whassqoodd ?
  Sentiment: positive

  Tweet: On the way to Malaysia...no internet access to Twit
  Sentiment: negative

  Tweet: is cleaning the house for her family who is comming later today..
  Sentiment: neutral

  Tweet: There won't another challenge really. Thanks.
  Sentiment: neutral

  Tweet: It's fun to go out dancing.
  Sentiment: positive

  Tw

In [ ]:
"""
Instructions:  
1. Using the bigscience/bloom-1b7 model we loaded in Part 2, generate task outputs for each of the 20 examples in the dataset
2. Save all the outputs and ground-truth labels to a dataframe
3. Compute the task performance, using the the provided ground-truth labels in the dataset, and the 
   evaluation metric you identified in Step 1.

Enter your code below:
"""

# already compute ground truth performance:
acc = correct/tot
print(f'{correct=}, {tot=}, {acc=}')
# not bad!

model.num_parameters()=1722408960
correct=14, tot=20, acc=0.7


In [ ]:
"""
Instructions:  
1. Load the bigscience/bloom-3b model and generate task outputs for each of the 20 examples in the dataset
2. Save all the outputs and ground-truth labels to a dataframe
3. Compute the task performance, using the the provided ground-truth labels in the dataset, and the 
   evaluation metric you identified in Step 1.


Enter your code below:
"""

# -- load bigscience/bloom-3b
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model_name = "bigscience/bloom-3b"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)

print(f'{model.num_parameters()=}')
tokenizer = AutoTokenizer.from_pretrained(model_name)

# -- generator
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

# -- eval model
correct = 0
tot = 0
for i, row in samples.iterrows():
  generated_text: str = generator(prompt_1(row['text']), max_new_tokens=1)[0]['generated_text']
  gen_ans = get_answer_from_fm(generated_text)
  print(f'\n\n --- Sample {i=}:')
  print(generated_text)
  print(f'{gen_ans=}')
  print(f"correct answer: {row['label_text']}")
  correct += int(gen_ans == row['label_text'])
  tot += 1

# - print performance
acc = correct/tot
print(f'{correct=}, {tot=}, {acc=}')

model.num_parameters()=3002557440


 --- Sample i=236:
Tweet: Sorry to anyone still stuck at Google Gaslight, where they repeatedly tell you it's the best place to work or whatever while they cull your colleagues by cutting off Corp access abruptly at 2AM
  Sentiment: negative

  Tweet:  think I'm going to succeed as a professor and, like, eventually get tenure. The first couple of conference and journal paper acceptances as junior faculty are a wild confidence booster
  Sentiment: positive

  Tweet: I saw a bird today at 4pm in the afernoon.
  Sentiment: neutral

  Tweet: juss came backk from Berkeleyy ; omg its madd fun out there havent been out there in a minute . whassqoodd ?
  Sentiment: positive

  Tweet: On the way to Malaysia...no internet access to Twit
  Sentiment: negative

  Tweet: is cleaning the house for her family who is comming later today..
  Sentiment: neutral

  Tweet: There won't another challenge really. Thanks.
  Sentiment: neutral

  Tweet: It's fun to go out da

Instructions:  
1. Compare the the quantitative performance of bloom-3B and bloom-1b7 parameter model.
2. Using a few of the outputs generated by models, describe a 1-2 qualitiative differences in the outputs 

**Answer (Provide your answer here):**

1.
1b7
correct=14, tot=20, acc=0.7
3b
correct=14, tot=20, acc=0.7

I don't know why they performed the same, I asked in discord it seems others sometimes get this behaviour? Perhaps it was my data set. 

I re-run it and printed the model size so I'm def running the right model:

model.num_parameters()=3002557440

correct=14, tot=20, acc=0.7

2. 
It's hard to identify a pattern given that the accuarcies match e.g. it would have been nicer if the two have difference acc and compare the diff e.g. maybe bloom3b gets some sentiment more than the other. But at least I found that an example on the same data point where they predict something different:

```
1b7
  Tweet: Me and mum are lost in grangemouth. Utter hilarity! Totally lost
  Sentiment: negative
gen_ans='negative'
correct answer: neutral
```
```
3b
  Tweet: Me and mum are lost in grangemouth. Utter hilarity! Totally lost
  Sentiment: positive
gen_ans='positive'
correct answer: neutral
```
both got the neutral wrong and predicted opposite sign. 

Ok on  --- Sample i=240: they predicted same mistake. Let's see if I can find another where they disagree but are both wrong. 

Ok I found one where 1b7 is right but 3b is wrong:
```
1b7
  Tweet: Must head back to the office
  Sentiment: neutral
gen_ans='neutral'
correct answer: neutral
```
```
3b
  Tweet: Must head back to the office
  Sentiment: negative
gen_ans='negative'
correct answer: neutral
```

Odd! But interesting. 

#### 2. Model Training Objectives
Here we will compare the performance of bloom-3b and [bloomz-3b](https://huggingface.co/bigscience/bloomz-3b).

Bloomz 3B is an *instruction-finetuned* model (a model explicitly trained to follow instructions), trained with a Bloom 3B backbone. 

* For more information on Bloom 3B , please see its [Model Card](https://huggingface.co/bigscience/bloomz-3b) (also linked above). 

* For more information on instruction fine-tuning, check out the corresponding [paper](https://arxiv.org/abs/2211.01786).

In [1]:
!pip install transformers torch datasets  accelerate bitsandbytes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import transformers
import torch
from datasets import load_dataset
from transformers import pipeline

# do assignment once
import random
import numpy as np

# -- load bigscience/bloomz-3b
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model_name = "bigscience/bloomz-3b"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)

print(f'{model.num_parameters()=}')
tokenizer = AutoTokenizer.from_pretrained(model_name)

# -- generator
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

In [5]:
"""
Instructions:  
1. Load the bigsciece/bloomz-3b model and generate task outputs for each of the 20 examples in the dataset
2. Save all the outputs and ground-truth labels to a dataframe
3. Compute the task performance, using the the provided ground truth labels in the dataset, and the 
  evaluation metric you identified in Step 1.


Enter your code below:
"""
# !pip install transformers torch datasets  accelerate bitsandbytes
import transformers
import torch
from datasets import load_dataset
from transformers import pipeline

# do assignment once
import random
import numpy as np

seed = 2
random.seed(seed)
np.random.seed(seed)

dataset = load_dataset("mteb/tweet_sentiment_extraction", split="test")

samples = dataset.to_pandas().sample(n=20)
sample_review = samples.iloc[0].text
print(f'{sample_review=}')

# --
def prompt_1(tweet):
  prompt = f'''Tweet: Sorry to anyone still stuck at Google Gaslight, where they repeatedly tell you it's the best place to work or whatever while they cull your colleagues by cutting off Corp access abruptly at 2AM
  Sentiment: negative

  Tweet:  think I'm going to succeed as a professor and, like, eventually get tenure. The first couple of conference and journal paper acceptances as junior faculty are a wild confidence booster
  Sentiment: positive

  Tweet: I saw a bird today at 4pm in the afernoon.
  Sentiment: neutral

  Tweet: juss came backk from Berkeleyy ; omg its madd fun out there havent been out there in a minute . whassqoodd ?
  Sentiment: positive

  Tweet: On the way to Malaysia...no internet access to Twit
  Sentiment: negative

  Tweet: is cleaning the house for her family who is comming later today..
  Sentiment: neutral

  Tweet: There won't another challenge really. Thanks.
  Sentiment: neutral

  Tweet: It's fun to go out dancing.
  Sentiment: positive

  Tweet: We will deliver you athletic greens. Information comming soon and will be sent to your email if you signed up.
  Sentiment: neutral

  Tweet: LOL. Playing soccer isn`t easy anymore compared when we were young
  Sentiment: negative

  Tweet: {tweet}
  Sentiment:'''
  return prompt

# -- eval model
def get_answer_from_fm(generated_text) -> str:
  gen_ans = generated_text.split('\n')[-1].split(' ')[-1]
  return gen_ans

correct = 0
tot = 0
for i, row in samples.iterrows():
  generated_text: str = generator(prompt_1(row['text']), max_new_tokens=1)[0]['generated_text']
  gen_ans = get_answer_from_fm(generated_text)
  print(f'\n\n --- Sample {i=}:')
  print(generated_text)
  print(f'{gen_ans=}')
  print(f"correct answer: {row['label_text']}")
  correct += int(gen_ans == row['label_text'])
  tot += 1

# - print performance
acc = correct/tot
print(f'{correct=}, {tot=}, {acc=}')

sample_review='My mobile has stopped recieving text messages AGAIN. Stoooopid thing'


 --- Sample i=236:
Tweet: Sorry to anyone still stuck at Google Gaslight, where they repeatedly tell you it's the best place to work or whatever while they cull your colleagues by cutting off Corp access abruptly at 2AM
  Sentiment: negative

  Tweet:  think I'm going to succeed as a professor and, like, eventually get tenure. The first couple of conference and journal paper acceptances as junior faculty are a wild confidence booster
  Sentiment: positive

  Tweet: I saw a bird today at 4pm in the afernoon.
  Sentiment: neutral

  Tweet: juss came backk from Berkeleyy ; omg its madd fun out there havent been out there in a minute . whassqoodd ?
  Sentiment: positive

  Tweet: On the way to Malaysia...no internet access to Twit
  Sentiment: negative

  Tweet: is cleaning the house for her family who is comming later today..
  Sentiment: neutral

  Tweet: There won't another challenge really. Thanks.
 

Instructions:  
1. Compare the the quantitative performance of bloom-3b and bloomz-3b.
2. Using a few of the outputs generated by models, describe 1-2 qualitiative differences in the outputs 

**Answer (Provide your answer here):**

1.
bloomz is worse:

correct=12, tot=20, acc=0.6

bloom was
model.num_parameters()=3002557440

correct=14, tot=20, acc=0.7

2.

bloom 2b
  Tweet: Me and mum are lost in grangemouth. Utter hilarity! Totally lost
  Sentiment: positive
gen_ans='positive'
correct answer: neutral

  Tweet: Me and mum are lost in grangemouth. Utter hilarity! Totally lost
  Sentiment: negative
gen_ans='negative'
correct answer: neutral

same mistake but opposite prediction. Interesting. 

bloom 3b
  Tweet:  you guys didn`t say hi or answer my questions yesterday  but nice songs.
  Sentiment: positive
gen_ans='positive'
correct answer: positive


  Tweet:  you guys didn`t say hi or answer my questions yesterday  but nice songs.
  Sentiment: negative
gen_ans='negative'
correct answer: positive

they disagree on this one! interesting. Interesting, this tweet could be neutral. Not responding is not a good thing. I personally get really mad about ghosting. So not sure if this is a good tweet to compare them actually. 

same with this one

bloomz

  Tweet: making german apple slice  nom nom nom
  Sentiment: negative
gen_ans='negative'
correct answer: positive

  Tweet: making german apple slice  nom nom nom
  Sentiment: positive
gen_ans='positive'
correct answer: positive

ambiguous to me. I suppose nom nom is that they like eating. Fine. Then the conclusion is that bloomz is worse at detecting sublties. 



---
#### Bonus for fun: ChatGPT

As an additional comparison, try the above evaluation with [ChatGPT](https://openai.com/blog/chatgpt/), a *closed-source* model trained by OpenAI with instruction-finetuning. 

For now, ChatGPT provides a free way to interact with "Open"AI's closed models. You can access it at [https://chat.openai.com/](https://chat.openai.com/) (you may need to create an account first).

How do the outputs of ChatGPT compare against the open-source BLOOM and BLOOMZ models?

---
#### Bonus for credit: ChatGPT does your homework
As an additional bonus, can you think of a way to get ChatGPT to complete this assignment?

In code or text blocks below, paste your input to ChatGPT, and the resulting output. 

Provide a short response describing your thoughts on using ChatGPT to complete this assignment, and any necessary changes to complete this assignment.

Additional bonus for actually running ChatGPT-generated output and completing this assignment in the code blocks below.


In [ ]:
### Enter your input prompts for ChatGPT:



In [ ]:
### Copy and paste the resulting ChatGPT output:



In [ ]:
### Describe your experience with ChatGPT, and any changes you'd make 
### to successfully complete this assignment with ChatGPT's outputs, 
### in the text box below:

In [ ]:
### Make any necessary changes to the output to successfully 
### run the code and complete this assignment in the code blocks below



### And that's it! Congrats on finishing your first assignment!

Remember to submit this completed notebook to Canvas to recieve credit for this assignment